In [1]:
from MyDataset import CifarDataModule

from art.project import ArtProject
from art.checks import CheckResultExists, CheckScoreExists, CheckScoreLessThan, CheckScoreGreaterThan
from art.steps import EvaluateBaseline, OverfitOneBatch, Overfit#, TransferLearning
from torchmetrics import Accuracy, Precision, Recall
import torch.nn as nn
from lightning.pytorch.callbacks import EarlyStopping

from MyDataset import CifarDataModule
from checks import CheckClassImagesExist, CheckLenClassNamesEqualToNumClasses
from steps import DataAnalysis
import math

In [2]:
%load_ext autoreload
%autoreload 2
from lightning import seed_everything
seed_everything(23)

Global seed set to 23


23

# Proper data analysis

In [3]:
project = ArtProject("Cifar100", CifarDataModule(batch_size=32))
project.add_step(DataAnalysis(), [
    CheckResultExists("number_of_classes"),
    CheckResultExists("class_names"),
    CheckResultExists("number_of_examples_in_each_class"),
    CheckResultExists("img_dimensions"),
    CheckClassImagesExist(),
    CheckLenClassNamesEqualToNumClasses()])
project.run_all()


Summary: 
Step: Data analysis, Model: , Passed: True. Results:



In [4]:
from torchmetrics import Accuracy
import torch.nn as nn
from art.steps import EvaluateBaseline

NUM_CLASSES = project.get_step(0).get_latest_run()["number_of_classes"]
accuracy_metric, ce_loss = Accuracy(task="multiclass", num_classes = NUM_CLASSES), nn.CrossEntropyLoss()
project.register_metrics([accuracy_metric, ce_loss])

In [5]:
from art.metrics import SkippedMetric
from models.baselines import MlBaseline, HeuristicBaseline, AlreadyExistingResNet20Baseline
from art.checks import CheckScoreExists
baselines = [HeuristicBaseline, MlBaseline, AlreadyExistingResNet20Baseline]
for baseline in baselines:
    project.add_step(
        step = EvaluateBaseline(baseline), 
        checks = [CheckScoreExists(metric=accuracy_metric)],
        skipped_metrics=[SkippedMetric(metric=ce_loss)]
    )


In [6]:
project.run_all()

Summary: 
Step: Data analysis, Model: , Passed: True. Results:

Step: Evaluate Baseline, Model: HeuristicBaseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.012299999594688416
Step: Evaluate Baseline, Model: MlBaseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.15649999678134918
Step: Evaluate Baseline, Model: AlreadyExistingResNet20Baseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.6883000135421753


In [7]:
from models.ResNet import ResNet18
from art.steps import CheckLossOnInit
from art.checks import CheckScoreCloseTo
from torch.cuda import is_available

EXPECTED_LOSS = -math.log(1/NUM_CLASSES)
print(EXPECTED_LOSS)

4.605170185988091


In [8]:
project.add_step(
        CheckLossOnInit(ResNet18),
        [CheckScoreCloseTo(metric=ce_loss,
                           value=EXPECTED_LOSS, rel_tol=0.1)]
    )

project.run_all()

Summary: 
Step: Data analysis, Model: , Passed: True. Results:

Step: Evaluate Baseline, Model: HeuristicBaseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.012299999594688416
Step: Evaluate Baseline, Model: MlBaseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.15649999678134918
Step: Evaluate Baseline, Model: AlreadyExistingResNet20Baseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.6883000135421753
Step: Check Loss On Init, Model: ResNet18, Passed: True. Results:
	MulticlassAccuracy-validate: 0.015399999916553497
	CrossEntropyLoss-validate: 5.087794780731201
Code of the following steps was changed: ResNet18_Check Loss On Init
 Rerun could be needed.


In [9]:
from art.steps import OverfitOneBatch
from art.checks import CheckScoreLessThan
project.add_step(OverfitOneBatch(ResNet18, number_of_steps=40),
            [CheckScoreLessThan(metric=ce_loss, value=0.05)])
project.run_all()

Summary: 
Step: Data analysis, Model: , Passed: True. Results:

Step: Evaluate Baseline, Model: HeuristicBaseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.012299999594688416
Step: Evaluate Baseline, Model: MlBaseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.15649999678134918
Step: Evaluate Baseline, Model: AlreadyExistingResNet20Baseline, Passed: True. Results:
	MulticlassAccuracy-validate: 0.6883000135421753
Step: Check Loss On Init, Model: ResNet18, Passed: True. Results:
	MulticlassAccuracy-validate: 0.015399999916553497
	CrossEntropyLoss-validate: 5.087794780731201
Step: Overfit One Batch, Model: ResNet18, Passed: True. Results:
	MulticlassAccuracy-train: 1.0
	CrossEntropyLoss-train: 0.00015361519763246179
Code of the following steps was changed: ResNet18_Check Loss On Init
 Rerun could be needed.


In [10]:
from art.steps import Overfit
from art.checks import CheckScoreGreaterThan

project.add_step(Overfit(ResNet18, max_epochs=10),
            [CheckScoreGreaterThan(metric=accuracy_metric, value=0.8)])

In [11]:
project.run_all()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Overfitting model


c:\Users\matri\anaconda3\envs\art\lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
Using cache found in C:\Users\matri/.cache\torch\hub\facebookresearch_semi-supervised-ImageNet1K-models_master
c:\Users\matri\anaconda3\envs\art\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\matri\anaconda3\envs\art\lib\site-packages\torchvision\models\_utils.py:223: 

Training: 0it [00:00, ?it/s]

c:\Users\matri\anaconda3\envs\art\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Validating overfitted model
Validating model ResNet18
Error while executing step Overfit!
Traceback (most recent call last):

  File "c:\Users\matri\anaconda3\envs\art\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x000001DF8D1F3050, file "c:\Users\matri\anaconda3\envs\art\lib\site-packages\ipykernel_launcher.py"...
           └ <function _run_code at 0x000001DF8D1FB400>

  File "c:\Users\matri\anaconda3\envs\art\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
         └ <code object <module> at 0x000001DF8D1F3050, file "c:\Users\matri\anaconda3\envs\art\lib\site-packages\ipy

c:\Users\matri\anaconda3\envs\art\lib\site-packages\lightning\pytorch\trainer\call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
c:\Users\matri\anaconda3\envs\art\lib\site-packages\lightning\pytorch\trainer\connectors\checkpoint_connector.py:149: UserWarning: `.validate(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.validate(ckpt_path='best')` to use the best model or `.validate(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(


ValueError: `.validate(ckpt_path="best")` is set but `ModelCheckpoint` is not configured to save the best model.

In [ ]:
from art.steps import TransferLearning
from art import NeptuneLogger

logger = NeptuneLogger(project_name="mmaecki/transfer-learning", api_token="XXX")
early_stopping = EarlyStopping('CrossEntropyLoss-validate', patience=6)
project.add_step(TransferLearning(ResNet18,
                                    freezed_trainer_kwargs={"max_epochs": 40,
                                                            "check_val_every_n_epoch": 2,
                                                            "callbacks": [early_stopping]},
                                    unfreezed_trainer_kwargs={"max_epochs": 50,
                                                            "check_val_every_n_epoch": 2,
                                                            "callbacks": [early_stopping]},
                                    keep_unfrozen=1,
                                    logger=logger
                                    ),
                    [CheckScoreGreaterThan(metric=accuracy_metric, value=0.70)])

In [ ]:
project.run_all()